In [1]:
# Import Libraries
import tweepy 
## Preprocessing 
import pandas as pd 
from collections import defaultdict

In [2]:
# initialize api instance

ACCESS_TOKEN = "1046284094118625280-el4Tzo2cL9n5n0LdkaU7ltJsthwg4f"
ACCESS_TOKEN_SECRET = "mz1xspQS9BSVtUgJQhnwN9k52L4mwWlyIMkP1ezHFl4Wc"
CONSUMER_KEY = "wWt1l2oSLyqkfcHnhCzrJG0FH"
CONSUMER_SECRET = "cWUCAJfh2uUYA5nIreLUwF0QkTkweJtonUnYGU51LSjPH50kU0"

In [3]:
#Connect to Twitter through the API 

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET) 
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
# Lat Long of few cities

NYC = {'lat': 40.712776, 'long': -74.005974}
ATLANTA = {'lat': 33.748997, 'long': -84.387985}
AUSTIN = {'lat': 30.267153, 'long': -97.743057}
MIAMI = {'lat': 25.761681, 'long': -80.191788}

woeid_NYC = api.closest_trends(**NYC)[0].get('woeid')
woeid_atlanta = api.closest_trends(**ATLANTA)[0].get('woeid')
woeid_austin = api.closest_trends(**AUSTIN)[0].get('woeid')
woeid_miami = api.closest_trends(**MIAMI)[0].get('woeid')

In [5]:
# Taking WOEID (Where On Earth ID provided by Yahoo.com) from previous result and fetching trending topics

nyc_trend = api.get_place_trends(woeid_nyc)
atlanta_trend = api.get_place_trends(woeid_atlanta)
austin_trend = api.get_place_trends(woeid_austin)
miami_trend = api.get_place_trends(woeid_miami)

NameError: name 'woeid_nyc' is not defined

In [6]:
MAX_TWEETS = 5000
RADIUS = "25mi"
COUNT_PER_PAGE = 100
tweets_for_cities = defaultdict(list)

def retrieve_and_store_tweets(search_query, lat_long, city_name, sector_name):
    tweets = tweepy.Cursor(api.search_tweets, q = search_query, geocode = lat_long, count = COUNT_PER_PAGE).items(MAX_TWEETS)
    
    # Pulling information from tweets iterable object
    # Add or remove tweet information you want in the below list comprehension
    tweets_list = [[tweet.text, tweet.created_at, tweet.id_str, tweet.favorite_count, tweet.user.screen_name, tweet.user.id_str, tweet.user.location, tweet.user.followers_count, tweet.coordinates, tweet.place] for tweet in tweets]

    # Creation of dataframe from tweets_list
    # Did not include column names to simplify code
    tweets_df = pd.DataFrame(tweets_list)
    
    tweets_for_cities[city_name].append(tweets_df)

In [61]:
sectors_and_topics = defaultdict(list)
sectors_and_topics["Public Safety"].extend(["Safety", "Public Safety", "Emergency", "Crime", "Theft", "Violence", "Gun Violence"])
sectors_and_topics["Healthcare"].extend(["Health", "Mental Health", "Healthcare"])
sectors_and_topics["Startup"].extend(["Startup", "Innovation"])
sectors_and_topics["Politics"].extend(["Politics", "Republicans", "Democrats"])
sectors_and_topics["Unemployment"].extend(["Unemployment", "Wages", "Low Wages", "Recession", "Inflation", "Layoffs"])
sectors_and_topics["Environment"].extend(["Pollution"])
sectors_and_topics["LGBTQ"].extend(["LGBTQ"])

cities = {"NewYork": NYC.values(), "Atlanta": ATLANTA.values(), "Austin": AUSTIN.values(), "Miami": MIAMI.values()}

for city, coordinates in cities.items():
    
    lat_long = ','.join(str(item) for item in coordinates) + "," + RADIUS
    retrieve_and_store_tweets(city, lat_long, city, "City")
    
    for industry, topics in sectors_and_topics.items():
        for topic in topics:
            retrieve_and_store_tweets(topic, lat_long, city, industry)

Rate limit reached. Sleeping for: 759
Rate limit reached. Sleeping for: 754
Rate limit reached. Sleeping for: 758
Rate limit reached. Sleeping for: 759
Rate limit reached. Sleeping for: 766
Rate limit reached. Sleeping for: 760
Rate limit reached. Sleeping for: 768
Rate limit reached. Sleeping for: 771


In [97]:
Columns = ["Text", "CreatedAt", "TweetId", "FavoriteCount", "ScreenName", "UserId", "UserLocation", "UserFollowers", "UserCoordinate", "Place"]

for city,list_of_tweets in tweets_for_cities.items():
    topic_count = 0
    for tweet_df in list_of_tweets:
        if len(tweet_df) > 0:
            topic_count += 1
            file_name = '~/Documents/Practice/TwitterData/{}/TweetsTopic{}.csv'.format(city, topic_count)
            tweet_df.to_csv(file_name, index = False, header=True)

In [67]:
len(tweets_for_cities["Austin"])

48

In [81]:
x=tweets_for_cities["Austin"][44][1:2]
x.columns = Columns

pandas.core.frame.DataFrame

In [82]:
Columns = ["Text", "CreatedAt", "TweetId", "FavoriteCount", "ScreenName", "UserId", "UserLocation", "UserFollowers", "UserCoordinate", "Place"]

x=tweets_for_cities["Austin"][44][1:2]
x.columns = Columns

In [83]:
x

,Text,CreatedAt,TweetId,FavoriteCount,ScreenName,UserId,UserLocation,UserFollowers,UserCoordinate,Place
1,@KamalaHarris Where’s the inflation reduction?,2022-10-23 04:20:08+00:00,1584036927233150977,0,jennifersmithtx,1420955751800098816,"Austin, TX",1318,None,None


In [95]:
tweets_for_cities["Austin"][24].to_csv('~/Documents/Practice/TwitterData/Austin/TweetsTopic1.csv', index = False, header=True)

In [19]:
sectors_and_topics = defaultdict(list)
sectors_and_topics["Public Safety"].extend(["Safety", "Public Safety", "Emergency", "Crime", "Theft", 
                                            "Violence", "Gun Violence", "Abscond", "Harassmemt", "SexualAbuse", 
                                            "Assault", "Bribe", "Burglar", "Offense", "Unlawful", "Forgery", 
                                            "Illegal", "Gambling", "Violation", "Homicide", "Imprisonment", 
                                            "Trafficking", "Extrotion", "Accident", "Manslaughter", "Criminal", 
                                            "Perjury", "Robbery", "Terrorism", "Terrorist", "NYC", "NewYork"])

for topic in sectors_and_topics["Public Safety"]:
    retrieve_and_store_tweets(topic, '40.712776,-74.005974,25mi', "NewYork", "Public Safety")


Rate limit reached. Sleeping for: 800
Rate limit reached. Sleeping for: 747
Rate limit reached. Sleeping for: 809


In [20]:
len(tweets_for_cities["NewYork"])

99

In [21]:
tweets_for_cities["NewYork"]

[Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Index: [],
 Empty DataFrame
 Columns: []
 Ind

In [26]:
Columns = ["Text", "CreatedAt", "TweetId", "FavoriteCount", "ScreenName", "UserId", "UserLocation", "UserFollowers", "UserCoordinate", "Place"]

for city,list_of_tweets in tweets_for_cities.items():
    topic_count = 0
    for tweet_df in list_of_tweets:
        if len(tweet_df) > 0:
            topic_count += 1
            file_name = '~/Documents/Practice/TwitterData/{}/PublicSafety/TweetsTopic{}.csv'.format(city, topic_count)
            tweet_df.to_csv(file_name, index = False, header=True)

In [81]:
import re
from textblob import TextBlob
def get_tweet_sentiment(tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        cleaned_tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet_text).split())
        analysis = TextBlob(cleaned_tweet)
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

nindex = []
topic1_data = pd.read_csv('~/Documents/Practice/TwitterData/NewYork/PublicSafety/TweetsTopic1.csv')
Columns = ["Text", "CreatedAt", "TweetId", "FavoriteCount", "ScreenName", "UserId", "UserLocation", "UserFollowers", "UserCoordinate", "Place"]
topic1_data.columns = Columns
for index, tweet_text in enumerate(topic1_data['Text']):
    if get_tweet_sentiment(tweet_text) == 'negative':
        negative_tweets.append(tweet_text)
        nindex.append(index)
        
ntweets_df = topic1_data.iloc[nindex]
ntweets_df.columns = Columns

file_name = '~/Documents/Practice/TwitterData/{}/PublicSafety/Topic1NegativeTweets.csv'.format(city)
ntweets_df.to_csv(file_name, index = False, header=True)